In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
okt = Okt()

In [2]:
row_data = pd.read_json('./playlists.json', encoding='utf-8')
row_data = row_data.iloc[:, 0]

In [3]:
row_data

0        {'id': 100005450, 'name': '발라드와 함께~~~', 'songs...
1        {'id': 100009421, 'name': '어디선가 들어본 팝송', 'song...
2        {'id': 100009811, 'name': '신나는 힙합과 댄스음악입니다.', ...
3        {'id': 100021337, 'name': '신나는 힙합과 댄스음악입니다.', ...
4        {'id': 100022115, 'name': '클럽 엔비 힙합 음악', 'song...
                               ...                        
54889    {'id': 482710204, 'name': '뉴트로계의 신성  신스웨이브 모음❤...
54890    {'id': 482710540, 'name': '매장 오픈할때 틀어놓기 편한 BGM...
54891    {'id': 482712084, 'name': '커피보다 에이드가 마시고픈 날', ...
54892    {'id': 482712481, 'name': '너도 알았으면 하는 20년 3월 노...
54893    {'id': 482716866, 'name': 'k_정리', 'songs': [31...
Name: data, Length: 54894, dtype: object

In [4]:
type( row_data.iloc[0]['songs'] )

list

In [5]:
row_data.iloc[0].keys()

dict_keys(['id', 'name', 'songs'])

In [6]:
title=[]
songs=[]
for PL in row_data:
    title.append(PL['name'])
    songs.append(PL['songs'])
data = pd.DataFrame({'title' : title, 'songs' : songs})

In [7]:
# 중복 타이틀인 경우 행 제거
title= []
songs=[]
for i in data['title'].drop_duplicates().index:
    title.append(data['title'].iloc[i])
    songs.append(data['songs'].iloc[i])
data = pd.DataFrame({'title' : title, 'songs' : songs})

In [8]:
data

,title,songs
0,발라드와 함께~~~,"[2397, 8305, 9433, 48957, 48958, 58173, 58506,..."
1,어디선가 들어본 팝송,"[19537, 325198, 331330, 331379, 335473, 347362..."
2,신나는 힙합과 댄스음악입니다.,"[104419, 105894, 106517, 290024, 317670, 32519..."
3,클럽 엔비 힙합 음악,"[286490, 327850, 522404, 651369, 774524, 83393..."
4,분위기 속 클럽음악,"[2307, 285721, 285963, 648118, 800208, 826455,..."
...,...,...
54529,뉴트로계의 신성 신스웨이브 모음❤✋,"[4338298, 5470835, 5804238, 30700525, 30785493..."
54530,매장 오픈할때 틀어놓기 편한 BGM,"[6676, 6677, 104083, 171606, 272495, 335616, 3..."
54531,커피보다 에이드가 마시고픈 날,"[8151903, 31319799, 31535725, 31667178, 318388..."
54532,너도 알았으면 하는 20년 3월 노래들,"[32421270, 32422375, 32422376, 32422483, 32422..."


In [9]:
i = 5
data['title'].iloc[i], okt.pos(data['title'].iloc[i], stem=True), okt.nouns(data['title'].iloc[i])

('멍해지고 싶은 날...',
 [('멍하다', 'Adjective'), ('싶다', 'Verb'), ('날', 'Noun'), ('...', 'Punctuation')],
 ['날'])

In [10]:
okt.pos('너도 알았으면 하는 20년 3월 노래들	.', stem=True)

[('너', 'Noun'),
 ('도', 'Josa'),
 ('알다', 'Verb'),
 ('하다', 'Verb'),
 ('20년', 'Number'),
 ('3월', 'Number'),
 ('노래', 'Noun'),
 ('들', 'Suffix'),
 ('\t', 'Foreign'),
 ('.', 'Punctuation')]

위의 결과를 통해 Adjective와 Noun만 뽑아오면 된다는 사실을 알게 된다

In [11]:
# 의미 있는 단어만을 위한 불용어 목록
# 장르, 무의미한 단어들을 직접 삽입하시면 됩니다.

# stopwords = [
#     '때',
#     '어디',
#     '이다',
#     '그',
#     '무조건',
#     '노래',
    
# ]

In [12]:
# 최소한의 불용어 목록

stopwords = [
    '때',
    '어디',
    '이다',
    '그'
]

In [13]:
PL_to_tokens = []
for i in range(data.shape[0]):
# for i in range(10):
    # tokens : 플레이리스트의 토큰으로 넣을 실질 token
    # row_tokens : 토큰화된 토큰들
    tokens = []
    row_tokens = okt.pos(data.iloc[i]['title'], stem=True)
    
    for token in row_tokens:
#         if ((token[1] != 'Josa') & (token[1] != 'Verb') & (token[1] != 'Suffix') & (token[1] != 'Punctuation') & (token[1] != 'Foreign') ) and token[0] not in stopwords:
        if ((token[1] == 'Noun') | (token[1] == 'Adjective')) and token[0] not in stopwords:
            tokens.append(token[0])
    
    # { '플레이리스트 제목 1 ' : 토큰들 1 , ... }
    PL_to_tokens.append(tokens)

In [14]:
len(PL_to_tokens), PL_to_tokens

(54534,
 [['발라드'],
  ['선', '팝송'],
  ['신나다', '힙합', '댄스', '음악'],
  ['클럽', '엔비', '힙합', '음악'],
  ['분위기', '속', '클럽', '음악'],
  ['멍하다', '날'],
  ['무조건', '노래'],
  ['분', '만', '락'],
  ['새벽', '삶', '사랑', '이야기'],
  ['그렇다', '이노', '정말', '좋다'],
  [],
  ['일본', '한국', '같다', '노래', '대결'],
  ['랩', '표현', '슬픔', '이별', '그리움'],
  [],
  ['일본드라마', '애니', '영화'],
  ['애니', '음악', '원피스'],
  ['서정', '감성', '귀가', '편안하다', '음악'],
  ['비', '날', '술'],
  ['아무'],
  ['듀엣'],
  ['몽', '베스트'],
  [],
  ['예수전도단', '캠퍼스', '워십', '베스트'],
  [],
  ['두루', '불의', '음악', '촌'],
  ['신나다', '재즈', '센스', '있다', '음악'],
  ['비', '듣기', '좋다', '우울하다', '몽환', '팝송'],
  ['를', '기억'],
  ['가요'],
  ['슬프다', '발라드', '모음'],
  ['일요일', '아침', '상쾌하다', '음악'],
  ['내', '귀', '감', '겨'],
  ['테리', '슬프다', '발라드'],
  ['영혼', '고음', '폭발', '노래방', '발라드'],
  ['추억', '새록새록', '우리', '열광하다', '여름', '댄스', '곡'],
  ['프랑스', '팝퓰러송', '샹송'],
  ['추억', '소환', '우리', '사랑', '인기가요'],
  ['신비', '아름답다', '켈틱', '음악'],
  ['의', '디바'],
  ['상쾌하다', '아침', '올드팝'],
  ['집합'],
  ['탱고', '외로움', '슬픔', '춤'],
  ['포르투', '온도', '드림', '

In [15]:
data['tokens'] = PL_to_tokens

In [16]:
data

,title,songs,tokens
0,발라드와 함께~~~,"[2397, 8305, 9433, 48957, 48958, 58173, 58506,...",[발라드]
1,어디선가 들어본 팝송,"[19537, 325198, 331330, 331379, 335473, 347362...","[선, 팝송]"
2,신나는 힙합과 댄스음악입니다.,"[104419, 105894, 106517, 290024, 317670, 32519...","[신나다, 힙합, 댄스, 음악]"
3,클럽 엔비 힙합 음악,"[286490, 327850, 522404, 651369, 774524, 83393...","[클럽, 엔비, 힙합, 음악]"
4,분위기 속 클럽음악,"[2307, 285721, 285963, 648118, 800208, 826455,...","[분위기, 속, 클럽, 음악]"
...,...,...,...
54529,뉴트로계의 신성 신스웨이브 모음❤✋,"[4338298, 5470835, 5804238, 30700525, 30785493...","[뉴트로계, 신성, 신스, 웨이브, 모음]"
54530,매장 오픈할때 틀어놓기 편한 BGM,"[6676, 6677, 104083, 171606, 272495, 335616, 3...","[매장, 오픈, 편하다]"
54531,커피보다 에이드가 마시고픈 날,"[8151903, 31319799, 31535725, 31667178, 318388...","[커피, 이드, 고프다, 날]"
54532,너도 알았으면 하는 20년 3월 노래들,"[32421270, 32422375, 32422376, 32422483, 32422...","[너, 노래]"


In [17]:
entire_tokens = set({})
for tokens in PL_to_tokens:
    entire_tokens.update(tokens)

In [18]:
entire_tokens = list(entire_tokens)

In [19]:
len(entire_tokens)

12726

In [20]:
token_to_PL = []
entire_songs = []
PL_cnt = []
PL_songs_cnt = []
cnt = 0

##################         주의 !!      #######################
#
# 아래는 잘 작동하는지 테스트하기위해 5개의 토큰만을 봅니다.
# 전체 데이터를 얻고 싶다면 주석 처리된 부분으로 바꿔주세요!

# for token in entire_tokens[:5]:
for token in entire_tokens:
    cnt += 1
    if cnt % 10 == 0: print(cnt)
    PL_list = []
    PL_songs = set({})
# for token in entire_tokens:
    for i in range(data.shape[0]):
        if token in data['tokens'].iloc[i]:
            PL_list.append(data['title'].iloc[i])
            PL_songs.update(data['songs'].iloc[i])
    token_to_PL.append(PL_list)
    entire_songs.append(PL_songs)
    PL_cnt.append(len(PL_list))
    PL_songs_cnt.append(len(PL_songs))

entire_songs = list(entire_songs)

In [21]:
token_DF = pd.DataFrame({'token' : entire_tokens[:len(token_to_PL)], 'playlist_title' : token_to_PL, 'songs' : entire_songs, 'PL_cnt' : PL_cnt, 'PL_songs_cnt' : PL_songs_cnt})

In [22]:
token_DF.iloc[0]['playlist_title']

['영화 속 재즈 OST 그 장면으로의 행복한 상상 (상시업데이트)',
 '행복한 배우 류혜영의 BGM',
 '신나게, 편하게, 행복하게. 160609. 목요일.',
 '가사 없이도 전해지는 행복한 기운의 음악들~',
 '나 없이 행복할 널 위해...이별 발라드',
 '행복하자!',
 '행복한 드라이브 시간에 어울리는 재즈',
 '행복한 하루를 위한 연주곡',
 '오늘 하루 더 행복하게, Happy mood song',
 '쒼나는 겨울댄스&캐롤 들으면서 행복한 클스마스 ~ ❤',
 '행복한 Merry Christmas!!!',
 '듣고만 있어도 행복해지는 멜로디 1',
 '아프기도하고행복하기도했지',
 '천사를 찾아 사바사바~ 행복했던 그 시절 90년대 히트곡 모음',
 '행복해지는 음악주문! "행복스웩" st용 프로필뮤직',
 '행복해지고 싶을때 꺼내듣는 내 삶의 배경음악들',
 '난 네가 행복했으면 좋겠어',
 '듣기만 해도 행복해지는 노래♡─♡',
 '활기차고 행복할때 나오는 드라마 OST',
 '행복한 금요일 기분을 더욱 좋게 해줄 플레이 리스트 :)',
 '우리아이가 행복해지는 피아노 클래식',
 '가을을 즐기게 해줄 행복한 연주곡들~',
 '행복한 지금 이 순간!!',
 '행복해지는 인디음악',
 '취향저격 행복한 팝송',
 '행복한 주말을 위한 음악',
 '행복한 주말에 가벼운 산책',
 '누워서 이 노래 들으면 행복해짐!',
 '언제나 행복한 크리스맛  캐롤',
 '♥행복한 크리스마스 White Christmas Jazz 연주곡♥',
 '행복하고 즐거운 크리스마스',
 '행복한 크리스마스를 위한 캐롤',
 '행복한 결혼식에 빠질 수 없는 플레이리스트',
 '들으면 행복해 지는 노래~~~',
 '단어만 들어도 행복한 금요일!! 기분전환 시켜 줄 플레이리스트!!!',
 '사랑을 시작 했다면 행복하게 들어볼 노래!!',
 '단어만 들어도 행복한 금요일! 취향저격할 플레이 리스트!!:)',
 '다시만나면 행복해졌음 좋겠어 너와 난',
 '행복한 하루에 

In [23]:
token_DF = token_DF.sort_values(['PL_cnt'], ascending=[False])

In [24]:
token_DF = token_DF.reset_index(drop=True)

In [25]:
token_DF

,token,playlist_title,songs,PL_cnt,PL_songs_cnt
0,행복하다,"[영화 속 재즈 OST 그 장면으로의 행복한 상상 (상시업데이트), 행복한 배우 류...","{30277632, 32833538, 622595, 31825926, 3027764...",133,3844
1,점,"[161030 [프롬의 낭만잡화점] 파일럿 선곡표, 161114 [프롬의 낭만잡화점...","{5656580, 419855, 3995666, 30220319, 3858467, ...",36,827
2,비올라,"[독주 악기로 승화한 비올라, 킴 카쉬카시안, 현악기 클래식(첼로 비올라 콘트라베이스)]","{31139585, 404358, 2478086, 5451282, 5451284, ...",2,65
3,시트,[(아무리 브렉시트라지만) 이 영국 노래를 들으면 안 된다는 법이 있나요?],"{7949581, 4123278, 2704668, 5473181, 8027294, ...",1,28
4,안타,"[취향안타는 봄 노래 , 나의 카페 봄 list]","{4754435, 30006916, 30169608, 30143498, 446708...",1,50


In [27]:
token_DF.to_csv("token_DF.csv")